**Realizado por**: José Javier Díaz González

**Correo**: alu0101128894@ull.edu.es

In [3]:
import Pkg; 
Pkg.add("GLPK")
Pkg.add("JuMP")

using JuMP, GLPK

   Resolving package versions...


  No Changes to `C:\Users\Kephril\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Kephril\.julia\environments\v1.8\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\Kephril\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Kephril\.julia\environments\v1.8\Manifest.toml`

## **Problema de rutas de vehículos con capacidades (CVRP) - Capacitated Vehicle Routing Problem**



In [5]:
# Número de vehiculos
m = 3
# Número de clientes
n = 5
# Capacidad de los vehiculos
Q = 10

# Costo de transporte
c = [0 2 3 1 4 3;
     2 0 1 3 2 3;
     3 1 0 5 3 5;
     1 3 5 0 2 2;
     4 2 3 2 0 2;
     3 3 5 2 2 0]

# Demanda de los clientes
d = [0 1 2 8 4 8]

# Definición del modelo
model = Model(GLPK.Optimizer)

@variable(model, x[1:n+1, 1:n+1, 1:m], Bin) # Definición de las variables de decisión
@variable(model, f[1:n+1, 1:n+1, 1:m] >= 0) # Cantidad de productos transportados desde el cliente i al cliente j

# Definición de la función objetivo
@objective(model, Min, sum(c[i,j]*x[i,j,k] for i in 1:n+1, j in 1:n+1, k in 1:m))

# Definición de las restricciones

@constraint(model, [i in 2:n+1], sum(x[i,j,k] for j in 1:n+1, k in 1:m) == 1) # Cada cliente debe ser visitado exactamente una vez
@constraint(model, [i in 2:n+1], sum(x[j,i,k] for j in 1:n+1, k in 1:m) == 1)

@constraint(model, [k in 1:m], sum(x[1,j,k] for j in 1:n+1) == 1) # Cada vehiculo debe iniciar y terminar en el deposito
@constraint(model, [k in 1:m], sum(x[j,1,k] for j in 1:n+1) == 1)

@constraint(model, [i in 2:n+1, j in 2:n+1, k in 1:m], f[i,j,k] <= Q*x[i,j,k]) # Cantidad de productos transportados desde el cliente i al cliente j

optimize!(model) # Solution del modelo

# Imprimir la solucion
println("Valor óptimo: ", objective_value(model))
println("Solución: ")
for k in 1:m
    println("Vehiculo: ", k)
    for i in 1:n+1
        for j in 1:n+1
            if value(x[i,j,k]) == 1
                println("[", i, ",", j, "]")
            end
        end
    end
end

# Imprimir la cantidad de productos transportados desde el cliente i al cliente j
println("\nCantidad de productos transportados desde el cliente i al cliente j")
for k in 1:m
    println("\nVehiculo: ", k)
    for i in 1:n+1
        for j in 1:n+1
            if value(x[i,j,k]) == 1
                println("[", i, ",", j, "]: ", value(f[i,j,k]))
            end
        end
    end
end

Valor óptimo: 0.0
Solución: 
Vehiculo: 1
[1,1]
[2,2]
[3,3]
[4,4]
[5,5]
[6,6]
Vehiculo: 2
[1,1]
Vehiculo: 3
[1,1]

Cantidad de productos transportados desde el cliente i al cliente j

Vehiculo: 1
[1,1]: 0.0
[2,2]: 0.0
[3,3]: 0.0
[4,4]: 0.0
[5,5]: 0.0
[6,6]: 0.0

Vehiculo: 2
[1,1]: 0.0

Vehiculo: 3
[1,1]: 0.0
